In [ ]:
from IPython.display import Audio, HTML
import re
import os

In [ ]:
meta='/workspace/espnet/egs2/aishell/asr1/data_zhaoli/test/wav.scp'
use2nd=False
ignore_english=False
reverse=False
cut_heads=5000

wavmaps={}
with open(meta) as f:
    for line in f:
        line=line.strip()
        if line == '':
            continue
        wavid, wavpath=line.split()
        wavmaps[wavid]=wavpath


In [ ]:
iid_text_wavpath

In [ ]:
wavmaps.keys()

In [ ]:
iid_text_wavpath=[]
with open('res.out') as f:
#with open('../uttsizesorted2') as f:
    for line in f.readlines()[cut_heads:]:
        line=line.strip()
        if line=='':
            continue
        iid, text, iid2, text2=line.split()
        wavpath=wavmaps[iid]
        if use2nd:
            wavpath=wavmaps[iid2]
            text=text2
        iid_text_wavpath.append((iid, text, wavpath))
if ignore_english:
    iid_text_wavpath=[items for items in iid_text_wavpath if re.search('[a-zA-Z]', items[1]) is None and len(items[1])>1]
else:
    iid_text_wavpath=[items for items in iid_text_wavpath if len(items[1])>1] 
if reverse:
    iid_text_wavpath.reverse()
labels=[]

In [ ]:
cur_idx = ""
label = ""
htm = """
<p id='abc'>Press "Next" to start</p>
<br>
<button onclick="set_value(1)" style="width:20%;">Next</button>
<button onclick="set_value(-1)" style="width:20%;">Prev</button>
<button onclick="label(1)" style="width:20%;background-color:yellow">SHIT</button>
<button onclick="label(0)" style="width:20%;background-color:green">Fine</button>
<button onclick="view(0)" style="width:20%;">View</button>

<script type="text/Javascript">
    len="""+str(len(iid_text_wavpath))+"""
    idx=-1
    function callback(out){
        var res = null;
        // if output is a print statement
        if(out['msg_type'] == "stream"){
            res = out['content']['text'];
        }
        // if output is a python object
        else if(out['msg_type'] === "pyout"){
            res = out.data["text/plain"];
        }
        // if output is a python error
        else if(out['msg_type'] == "pyerr"){
            res = out.ename + ": " + out.evalue;
        }
        // if output is something we haven't thought of
        else{
            res = "[out type not implemented]";   
        }
        document.getElementById("abc").innerHTML=res
    }
    
    function label(yn){
        var command = "label = " + yn.toString();
        var kernel = IPython.notebook.kernel;
        kernel.execute(command);
        IPython.notebook.execute_cells([7])
    }
    
    function set_value(inc){
        if(inc>0 && idx>=len){
            document.getElementById("abc").innerHTML="End"
        }
        else if(inc<0 && idx<0){
            document.getElementById("abc").innerHTML="Head"
        }
        else{
            idx+=inc
            var command = "cur_idx = " + idx.toString();
            var kernel = IPython.notebook.kernel;
            kernel.execute(command);
            IPython.notebook.execute_cells([6, 8])
            
            kernel.execute('print(iid_text_wavpath[cur_idx][0])', {'iopub': {"output": callback}});
        }
    }
    
    function view(){
        IPython.notebook.execute_cells([9])
    }
</script>
"""
HTML(htm)

In [ ]:
print(iid_text_wavpath[cur_idx][1])
print(iid_text_wavpath[cur_idx][2], os.path.exists(iid_text_wavpath[cur_idx][2]))
Audio(iid_text_wavpath[cur_idx][2])

In [ ]:
labels.append((iid_text_wavpath[cur_idx][0], label))
print(f'labeled {iid_text_wavpath[cur_idx][0]}')
if len(labels)!=cur_idx+1:
    print('smth missing')

In [ ]:
if len(labels)==cur_idx:
    print('sfsg, follow above instr')
elif len(labels)>cur_idx:
    print(f'{len(labels)-cur_idx} ahead to go')
else:
    print('smth missing')

In [ ]:
labels

In [ ]:
labels=[]
print(labels)

In [ ]:
with open('label_res.out', 'w') as f:
    for iid, l in labels:
        f.write(f'{iid} {l}\n')